This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [1]:
from pydrake.all import ModelVisualizer, StartMeshcat, PackageMap

from manipulation import running_as_notebook

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://4e4bcb41-cafd-436c-bda4-33dbe7202b60.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


# Robot arms

The next two cells will give you a simple interface to move the joints around on a variety of robot arm models.  I highly recommend opening the "controls" menu in the meshcat visualizer. You can expand the tree under "drake", then check and uncheck the different properties (illustration / inertia / proximity) to explore the model (and the visualization tool).

Have a favorite robot that I haven't included here?  If you send me a link to a properly licensed URDF or SDF description of the robot and it's mesh files, I'm happy to add it!  It's worth taking a look at the files quickly, to get a sense for how they work: [SDF](https://github.com/RobotLocomotion/drake/blob/master/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf), [URDF](https://github.com/RobotLocomotion/drake/blob/master/manipulation/models/iiwa_description/urdf/iiwa14_no_collision.urdf).

Note: The Jaco doesn't visualize properly in this renderer yet.  See drake issue [#13846](https://github.com/RobotLocomotion/drake/issues/13846).

In [10]:
# First pick your robot by un-commenting one of these:
# robot = "Kuka LBR iiwa 7"
# robot = "Kuka LBR iiwa 14"
# robot = "Kinova Jaco Gen2 (7 DoF)"
robot = "Franka Emika Panda"
# robot = "UR3e"


def get_model_url(description):
    # Note: I could download remote model resources here if necessary.
    if description == "Kuka LBR iiwa 7":
        return "package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf"
    elif description == "Kuka LBR iiwa 14":
        return "package://drake/manipulation/models/iiwa_description/urdf/iiwa14_primitive_collision.urdf"
    elif description == "Kinova Jaco Gen2 (7 DoF)":
        return "package://drake/manipulation/models/jaco_description/urdf/j2s7s300.urdf"
    elif description == "Franka Emika Panda":
        return "package://drake/manipulation/models/franka_description/urdf/panda_arm_hand.urdf"
    elif description == "UR3e":
        return "package://drake/manipulation/models/ur3e/ur3e_cylinders_collision.urdf"
    raise Exception("Unknown model")


visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(url=get_model_url(robot))

visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


# Robot hands

I don't have the same smörgåsbord of robot models to offer for robot hands (yet).  We do have the allegro hand model available, and I'm happy to add more here (though many will require some care to handle their kinematic constraints).  For now, you can at least try out the [Allegro Hand](http://www.wonikrobotics.com/Allegro-Hand.htm).

TODO(russt): Consider porting Robotiq, Sandia, or IHY-hand from our [openhumanoids](https://github.com/openhumanoids/oh-distro/tree/master/software/models/common_components) project.

In [11]:
meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(
    url="package://drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf"
)

visualizer.Run(loop_once=not running_as_notebook)

meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


# Robots description files from other repositories

We've tried to make it very easy to pull in resources from other repositories.  Here's an example of pulling in files directly from github. Drake's MuJoCo parser is the least used/complete of our parsers (I only added a minimal version of it quite recently), so many of these don't parse beautifully yet -- I'm working on it! But it's a good example of how to pull in resources from other repositories, and if there is a particular model you need that doesn't parse well for you, you can let me know.

## MuJoCo Menagerie

In [12]:
if running_as_notebook:  # I don't want to download the menagerie in CI.
    meshcat.Delete()
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().package_map().AddRemote(
        package_name="mujoco_menagerie",
        params=PackageMap.RemoteParams(
            # This repository doesn't have up-to-date tags/releases; the scary
            # hash in the url is the most recent commit sha at the time of my
            # writing.
            urls=[
                f"https://github.com/google-deepmind/mujoco_menagerie/archive/0c8c9315506dbd4e9b3c1a6ff6faa28612792d1d.tar.gz"
            ],
            sha256=(
                "ea7e63453236c1e8583b3dc7acc75d9204ef8b9b8407d75c2c6a0b75e0be202a"
            ),
            strip_prefix="mujoco_menagerie-0c8c9315506dbd4e9b3c1a6ff6faa28612792d1d/",
        ),
    )
    visualizer.AddModels(
        url="package://mujoco_menagerie/kuka_iiwa_14/iiwa14.xml"
    )
    visualizer.Run(loop_once=True)
    meshcat.DeleteAddedControls()

## DeepMind Control Suite

In [6]:
if running_as_notebook:  # I don't want to download the dm_control suite in CI.
    meshcat.Delete()
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().package_map().AddRemote(
        package_name="dm_control",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/google-deepmind/dm_control/archive/refs/tags/1.0.15.tar.gz"
            ],
            sha256=(
                "bac091b18689330a99b7c18ddf86baa916527f5e4ab8e3ded0c8caff1dab2048"
            ),
            strip_prefix="dm_control-1.0.15/",
        ),
    )
    visualizer.AddModels(url="package://dm_control/dm_control/suite/dog.xml")
    plant = visualizer.parser().plant()
    visualizer.Run(loop_once=not running_as_notebook)
    meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4e4bcb41-cafd-436c-bda4-33dbe7202b60' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>